In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'
                       .format(postgres_user, postgres_pw,postgres_host, 
                        postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation', con=engine)

engine.dispose()

1. Determine all the variable types and find the fraction of the missing values for each variable.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

In [13]:
for col in df.columns:
    null_percent = df[col].isnull().sum()*100/len(df[col])
    print('{} percent null values in {}'.format(null_percent, col))

0.0 percent null values in PRIMARY_KEY
0.0 percent null values in STATE
0.0 percent null values in YEAR
17.62734584450402 percent null values in ENROLL
14.20911528150134 percent null values in TOTAL_REVENUE
14.20911528150134 percent null values in FEDERAL_REVENUE
14.20911528150134 percent null values in STATE_REVENUE
14.20911528150134 percent null values in LOCAL_REVENUE
14.20911528150134 percent null values in TOTAL_EXPENDITURE
14.20911528150134 percent null values in INSTRUCTION_EXPENDITURE
14.20911528150134 percent null values in SUPPORT_SERVICES_EXPENDITURE
17.62734584450402 percent null values in OTHER_EXPENDITURE
14.20911528150134 percent null values in CAPITAL_OUTLAY_EXPENDITURE
11.595174262734584 percent null values in GRADES_PK_G
8.847184986595174 percent null values in GRADES_KG_G
8.780160857908847 percent null values in GRADES_4_G
8.780160857908847 percent null values in GRADES_8_G
8.780160857908847 percent null values in GRADES_12_G
8.780160857908847 percent null values in 

2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?


We should avoid dropping data, but out of the above columns it makes the most sense to drop the points that have few null values. For example the sets of values that are less than 10% null. 

Because all the values come from the same year, it would make sense to use the mean as a replacement for missing values. This could be done for the columns that are missing less than 14-20% of the data. 

For the columns where we are missing close to 65% of the data it doesn't make as much sense to replace the values using the mean because we are missing most of the data. Instead we could limit the scope by dropping these missing values, or instead try to interpolate based on the similarity of other values in the row. 

3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using *only* the observations for that specific year.


In [108]:
df_cpy = df.copy()

cols = list(df_cpy.columns)

cols = [x for x in cols if x not in ('PRIMARY_KEY', 'STATE', 'YEAR')]

unique_yrs = df_cpy['YEAR'].unique()


for year in unique_yrs:
    for column in cols:       
        df_cpy[column].loc[df_cpy['YEAR'] == year] = df_cpy[column].loc[df_cpy['YEAR'] == year].fillna(df_cpy[column].loc[df_cpy['YEAR'] == year].mean())
          

for col in df_cpy.columns:
    null_percent = df_cpy[col].isnull().sum()*100/len(df_cpy[col])
    print('{} percent null values in {}'.format(null_percent, col))


0.0 percent null values in PRIMARY_KEY
0.0 percent null values in STATE
0.0 percent null values in YEAR
7.171581769436997 percent null values in ENROLL
3.418230563002681 percent null values in TOTAL_REVENUE
3.418230563002681 percent null values in FEDERAL_REVENUE
3.418230563002681 percent null values in STATE_REVENUE
3.418230563002681 percent null values in LOCAL_REVENUE
3.418230563002681 percent null values in TOTAL_EXPENDITURE
3.418230563002681 percent null values in INSTRUCTION_EXPENDITURE
3.418230563002681 percent null values in SUPPORT_SERVICES_EXPENDITURE
7.171581769436997 percent null values in OTHER_EXPENDITURE
3.418230563002681 percent null values in CAPITAL_OUTLAY_EXPENDITURE
6.836461126005362 percent null values in GRADES_PK_G
6.836461126005362 percent null values in GRADES_KG_G
6.836461126005362 percent null values in GRADES_4_G
6.836461126005362 percent null values in GRADES_8_G
6.836461126005362 percent null values in GRADES_12_G
6.836461126005362 percent null values in G

4. This time, fill in the missing values using interpolation (extrapolation).


In [110]:
df_cpy = df.copy()

cols = list(df_cpy.columns)

cols = [x for x in cols if x not in ('PRIMARY_KEY', 'STATE', 'YEAR')]

unique_yrs = df_cpy['YEAR'].unique()

for column in cols:       
    df_cpy[column] = df_cpy[column].interpolate()
          

for col in df_cpy.columns:
    null_percent = df_cpy[col].isnull().sum()*100/len(df_cpy[col])
    print('{} percent null values in {}'.format(null_percent, col))

0.0 percent null values in PRIMARY_KEY
0.0 percent null values in STATE
0.0 percent null values in YEAR
2.4128686327077746 percent null values in ENROLL
0.0 percent null values in TOTAL_REVENUE
0.0 percent null values in FEDERAL_REVENUE
0.0 percent null values in STATE_REVENUE
0.0 percent null values in LOCAL_REVENUE
0.0 percent null values in TOTAL_EXPENDITURE
0.0 percent null values in INSTRUCTION_EXPENDITURE
0.0 percent null values in SUPPORT_SERVICES_EXPENDITURE
2.4128686327077746 percent null values in OTHER_EXPENDITURE
0.0 percent null values in CAPITAL_OUTLAY_EXPENDITURE
0.0 percent null values in GRADES_PK_G
0.0 percent null values in GRADES_KG_G
0.0 percent null values in GRADES_4_G
0.0 percent null values in GRADES_8_G
0.0 percent null values in GRADES_12_G
0.0 percent null values in GRADES_1_8_G
0.0 percent null values in GRADES_9_12_G
0.0 percent null values in GRADES_ALL_G
0.0 percent null values in AVG_MATH_4_SCORE
0.0 percent null values in AVG_MATH_8_SCORE
0.0 percent n

5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?


Interpolation provided the fewest null values. All columns other than ENROLL, OTHER EXPENDITURE and AVG READING 8 SCORE had a 0 percent null value after interpolation. This was a significant improvement over filling year by year but just because there are fewer null values does not mean that the quality of the data is better. 